### Handoffs
A way for an agent to invoke another agent

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from agents import Agent, Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

tutorial_generator = Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and an outline, your job is to generate code snippets for each section of the outline."
        "Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples."
        "Where it makes sense, include comments in the code snippets to further explain the code."
    ),
    output_type=Tutorial
)

outline_builder = Agent(
    name="Outline Builder",
    instructions=(
        "Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline."
        "After making the outline, hand it to the tutorial generator agent."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(outline_builder, "Loops in Java")
print(tutorial_response.final_output)

Here's an outline for a tutorial on Loops in Java:

### Introduction
- Overview of loops in programming
- Importance and use cases of loops

### Types of Loops in Java
1. **For Loop**
   - Syntax
   - Simple example
   - Use cases and benefits

2. **Enhanced For Loop (For-Each Loop)**
   - Syntax
   - Example with arrays and collections
   - When to use enhanced for loop

3. **While Loop**
   - Syntax
   - Basic example
   - Use cases for while loop

4. **Do-While Loop**
   - Syntax
   - Example and difference from while loop
   - Appropriate use cases

### Nested Loops
- Explanation and example
- Performance considerations
- Common use cases for nested loops

### Loop Control Statements
1. **Break Statement**
   - Syntax and example
   - Use cases for breaking out of loops

2. **Continue Statement**
   - Syntax and example
   - Use cases for skipping iterations

3. **Return in Loops**
   - How return statement works within loops

### Best Practices
- Avoiding infinite loops
- Minimizi

In [3]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [4]:
result = await Runner.run(triage_agent, "How do I add 2 and 2?")
result.final_output

Handing off to math tutor agent


"Adding 2 and 2 is a straightforward process. Here's how you do it:\n\n1. **Identify the Numbers**: You have two numbers, which are both 2.\n\n2. **Add the First Number**: Start with the first 2.\n\n3. **Add the Second Number**: Add the second 2 to the first.\n\n    \\[\n    2 + 2 = 4\n    \\]\n\n4. **Result**: The sum of 2 and 2 is 4.\n\n**Example**: If you have two apples and you get two more apples, you now have four apples in total.\n\nThis simple process is the basic concept of addition!"

In [5]:
result = await Runner.run(triage_agent, "How did WW2 start?")
result.final_output

Handing off to history tutor agent


"World War II began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This aggressive action prompted Britain and France to declare war on Germany two days later, on September 3.\n\nTo understand the start of World War II, it's important to consider several key factors and events leading up to this moment:\n\n1. **Treaty of Versailles (1919):** The harsh terms imposed on Germany after World War I created economic hardship and resentment among Germans. The treaty required Germany to accept responsibility for the war, make substantial territorial concessions, and pay reparations.\n\n2. **Rise of Hitler and the Nazi Party:** Adolf Hitler and the National Socialist German Workers' Party (Nazi Party) capitalized on economic woes, nationalistic fervor, and dissatisfaction with the Weimar Republic to gain power. Hitler became Chancellor of Germany in 1933 and quickly established a totalitarian regime.\n\n3. **Expansionist Policies:** Hitler sought to overturn the Treaty

In [6]:
from agents import function_tool

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

result = await Runner.run(customer_service_agent, "Hello how much are tickets?")
print(result.final_output)

Could you please specify which event or location you're inquiring about?


#### Recommended Prompt Prefix
The recommended prompt prefix is a constant that OpenAI recommends you give to handoff agents to improve understanding.

In [7]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

billing_agent = Agent(
    name="Billing agent",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    <Fill in the rest of your prompt here>.""",
)